<a href="https://colab.research.google.com/github/karthickrajas/Lumos/blob/master/Graph_representation_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy
!pip install tensorflow
!pip install spektral
!!pip install --user scipy==1.8.1



import numpy as np
import tensorflow as tf
import spektral
# import spacy 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
cora_dataset = spektral.datasets.citation.Citation(name='cora')
test_mask = cora_dataset.mask_te
train_mask = cora_dataset.mask_tr
val_mask = cora_dataset.mask_va
graph = cora_dataset.graphs[0]
features = graph.x
adj = graph.a
labels = graph.y

/usr/local/lib/python3.8/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [13]:
adj = adj.todense()+np.eye(adj.shape[0])
adj = adj.astype('float32')

print(features.shape, '\n', adj.shape, '\n', labels.shape)

(2708, 1433) 
 (2708, 2708) 
 (2708, 7)


In [14]:
print(np.sum(train_mask)), print(np.sum(val_mask)), print(np.sum(test_mask))

140
500
1000


(None, None, None)

In [16]:
def masked_softmax_cross_entropy(logits, labels, mask):
  loss = tf.nn.softmax_cross_entropy_with_logits(logits =logits, labels = labels)
  mask = tf.cast(mask, dtype=tf.float32)
  mask /= tf.reduce_mean(mask)
  loss *= mask
  return tf.reduce_mean(loss)

def masked_accuracy(logits, labels, mask):
  correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
  accuracy_all = tf.cast(correct_prediction, tf.float32)
  mask = tf.cast(mask, dtype= tf.float32)
  mask /= tf.reduce_mean(mask)
  accuracy_all *= mask
  return tf.reduce_mean(accuracy_all)


In [17]:
def gnn(fts, adj, transform, activation):
  seq_fts = transform(fts)
  ret_fts = tf.matmul(adj, seq_fts)
  return activation(ret_fts)

In [20]:
def train_cora(fts, adj, gnn_fn, units , epochs, lr):
  lyr_1 = tf.keras.layers.Dense(units)
  lyr_2 = tf.keras.layers.Dense(7)

  def cora_gnn(fts, adj):
    hidden = gnn_fn(fts, adj, lyr_1, tf.nn.relu)
    logits = gnn_fn(hidden, adj, lyr_2, tf.identity)
    return logits

  optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

  best_accuracy = 0.0
  for ep in range(epochs+1):
    with tf.GradientTape() as t:
      logits = cora_gnn(fts, adj)
      loss = masked_softmax_cross_entropy(logits, labels, train_mask)
    
    variables = t.watched_variables()
    grads = t.gradient(loss, variables)
    optimizer.apply_gradients(zip(grads, variables))

    logits = cora_gnn(fts, adj)
    val_accuracy = masked_accuracy(logits, labels, val_mask)
    test_accuracy = masked_accuracy(logits, labels, test_mask)

    if val_accuracy > best_accuracy:
      best_accuracy = val_accuracy
      print('Epoch', ep, '| Training loss: ',loss.numpy(), '| Val accuracy', val_accuracy.numpy(),
            '|Test accuracy:', test_accuracy.numpy())


In [21]:
train_cora(features, adj, gnn, 32, 200, 0.01)

Epoch 0 | Training loss:  5.831041 | Val accuracy 0.326 |Test accuracy: 0.30200002
Epoch 1 | Training loss:  4.0670323 | Val accuracy 0.36799997 |Test accuracy: 0.37
Epoch 2 | Training loss:  3.065825 | Val accuracy 0.39 |Test accuracy: 0.39399996
Epoch 4 | Training loss:  1.4544667 | Val accuracy 0.52 |Test accuracy: 0.516
Epoch 5 | Training loss:  0.90851533 | Val accuracy 0.638 |Test accuracy: 0.632
Epoch 6 | Training loss:  0.7184535 | Val accuracy 0.67800003 |Test accuracy: 0.69
Epoch 7 | Training loss:  0.5841163 | Val accuracy 0.704 |Test accuracy: 0.702
Epoch 8 | Training loss:  0.5076722 | Val accuracy 0.74399996 |Test accuracy: 0.72499996
Epoch 9 | Training loss:  0.40341625 | Val accuracy 0.7559999 |Test accuracy: 0.73399997
Epoch 46 | Training loss:  0.06302718 | Val accuracy 0.75799996 |Test accuracy: 0.76100004
Epoch 62 | Training loss:  0.008255297 | Val accuracy 0.758 |Test accuracy: 0.7569999
Epoch 63 | Training loss:  0.007970731 | Val accuracy 0.76000005 |Test accura

In [22]:
train_cora(features, tf.eye(adj.shape[0]), gnn, 32, 200, 0.01)

Epoch 0 | Training loss:  1.9562379 | Val accuracy 0.278 |Test accuracy: 0.248
Epoch 1 | Training loss:  1.6789219 | Val accuracy 0.39599997 |Test accuracy: 0.349
Epoch 2 | Training loss:  1.4422939 | Val accuracy 0.44800004 |Test accuracy: 0.40199998
Epoch 3 | Training loss:  1.1920819 | Val accuracy 0.478 |Test accuracy: 0.441
Epoch 4 | Training loss:  0.94192517 | Val accuracy 0.494 |Test accuracy: 0.465
Epoch 5 | Training loss:  0.7149641 | Val accuracy 0.50200003 |Test accuracy: 0.48299995
Epoch 6 | Training loss:  0.52641535 | Val accuracy 0.52199996 |Test accuracy: 0.49699998
Epoch 8 | Training loss:  0.2781419 | Val accuracy 0.534 |Test accuracy: 0.506
Epoch 9 | Training loss:  0.2055399 | Val accuracy 0.546 |Test accuracy: 0.516
Epoch 10 | Training loss:  0.1551987 | Val accuracy 0.548 |Test accuracy: 0.5149999
Epoch 11 | Training loss:  0.11979565 | Val accuracy 0.54999995 |Test accuracy: 0.51799995
Epoch 14 | Training loss:  0.060455266 | Val accuracy 0.55399996 |Test accura

In [23]:
deg = tf.reduce_sum(adj, axis=-1)
train_cora(features, adj/deg, gnn, 32, 200, 0.01)

Epoch 0 | Training loss:  1.9493388 | Val accuracy 0.364 |Test accuracy: 0.39599997
Epoch 1 | Training loss:  1.7641393 | Val accuracy 0.52 |Test accuracy: 0.548
Epoch 2 | Training loss:  1.5430015 | Val accuracy 0.60400003 |Test accuracy: 0.62700003
Epoch 3 | Training loss:  1.3089193 | Val accuracy 0.664 |Test accuracy: 0.70699996
Epoch 4 | Training loss:  1.0944315 | Val accuracy 0.716 |Test accuracy: 0.749
Epoch 5 | Training loss:  0.90117234 | Val accuracy 0.75200003 |Test accuracy: 0.78
Epoch 6 | Training loss:  0.7308053 | Val accuracy 0.76000005 |Test accuracy: 0.79199994
Epoch 7 | Training loss:  0.58567417 | Val accuracy 0.77199996 |Test accuracy: 0.79399997
Epoch 8 | Training loss:  0.46526882 | Val accuracy 0.784 |Test accuracy: 0.79599994


In [26]:
norm_deg = tf.linalg.diag(1.0 / tf.sqrt(deg))
norm_adj = tf.matmul(norm_deg, tf.matmul(adj, norm_deg))
train_cora(features, norm_adj, gnn, 32, 200, 0.01)

Epoch 0 | Training loss:  1.9500102 | Val accuracy 0.42999998 |Test accuracy: 0.47
Epoch 1 | Training loss:  1.7713166 | Val accuracy 0.54599994 |Test accuracy: 0.6
Epoch 2 | Training loss:  1.5699306 | Val accuracy 0.61399996 |Test accuracy: 0.641
Epoch 3 | Training loss:  1.350128 | Val accuracy 0.662 |Test accuracy: 0.68299997
Epoch 4 | Training loss:  1.1366434 | Val accuracy 0.698 |Test accuracy: 0.72999996
Epoch 5 | Training loss:  0.93799603 | Val accuracy 0.728 |Test accuracy: 0.7569999
Epoch 6 | Training loss:  0.75983226 | Val accuracy 0.74599993 |Test accuracy: 0.77199996
Epoch 7 | Training loss:  0.60448736 | Val accuracy 0.76799995 |Test accuracy: 0.7839999
Epoch 8 | Training loss:  0.47458473 | Val accuracy 0.7799999 |Test accuracy: 0.79599994
Epoch 9 | Training loss:  0.36880937 | Val accuracy 0.78 |Test accuracy: 0.80099994
Epoch 10 | Training loss:  0.2846635 | Val accuracy 0.782 |Test accuracy: 0.80399996
Epoch 12 | Training loss:  0.16854568 | Val accuracy 0.784 |Tes